## Introduction

Using this [dataset](https://components.one/datasets/pitchfork-reviews-dataset/) we'll build a recommendation system which returns the top 5 albums most similar to the given one. This borrows from [a tutorial](https://towardsdatascience.com/how-to-build-from-scratch-a-content-based-movie-recommender-with-natural-language-processing-25ad400eb243) which aims to build a movie content-based recommendation system using NLP methods.

## Setup
First, let's make sure we're in the correct working directory and have all the necessary packages imported.
We'll also read in the dataset--it's been cleaned beforehand on my local machine; a simple process to get it into a csv rather than the SQLLite db I had downloaded.
Finally, we'll set a flag to indicate which vectorizer we are using in the interest of testing both and comparing (A/B testing).


In [15]:
%cd "drive/My Drive/Colab Notebooks"
!pwd

[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks'
/Users/tejassiripurapu/Google Drive/Colab Notebooks
/Users/tejassiripurapu/Google Drive/Colab Notebooks


In [16]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [17]:
df = pd.read_csv("data/pitchfork_reviews_cleaned.csv")
print(df.head())

tfidf = True # if True: use TfidfVectorizer, else: CountVectorizer
if tfidf:
    print("USING TFIDF-VECTORIZER")
else:
    print("USING COUNT-VECTORIZER")

             artist                                       album       genre  \
0       David Byrne  “…The Best Live Show of All Time” — NME EP        Rock   
1         DJ Healer           Lost Lovesongs / Lostsongs Vol. 2  Electronic   
2       Jorge Velez                                 Roman Birds  Electronic   
3           Chandra                          Transportation EPs        Rock   
4  The Chainsmokers                                    Sick Boy  Electronic   

   score             author                                             review  
0    5.5          Andy Beta  Viva Brother, Terris, Mansun, the Twang, Joe L...  
1    6.2        Chal Ravens  The Prince of Denmark—that is, the proper prin...  
2    7.9   Philip Sherburne  Jorge Velez has long been prolific, but that’s...  
3    7.8          Andy Beta  When the Avalanches returned in 2016 after an ...  
4    3.1  Larry Fitzmaurice  We’re going to be stuck with the Chainsmokers ...  


## A Little Preprocessing
Now that our data is loaded into a dataframe, let's separate out the written review and use the album title as the index.

NOTE: The set we're working with may be too big to handle atm (session is crashing when trying to vectorize). Randomly sample 8000 reviews to begin with.

In [18]:
working_df = df[['artist', 'album', 'review']].copy()
# working_df.set_index('album', inplace=True)
print(working_df.head())
print(len(working_df))
working_df.dropna(inplace=True, how='any')
print(len(working_df)) # How many na rows were there? 5 (which ones?)
sample_df = working_df.sample(8000)

             artist                                       album  \
0       David Byrne  “…The Best Live Show of All Time” — NME EP   
1         DJ Healer           Lost Lovesongs / Lostsongs Vol. 2   
2       Jorge Velez                                 Roman Birds   
3           Chandra                          Transportation EPs   
4  The Chainsmokers                                    Sick Boy   

                                              review  
0  Viva Brother, Terris, Mansun, the Twang, Joe L...  
1  The Prince of Denmark—that is, the proper prin...  
2  Jorge Velez has long been prolific, but that’s...  
3  When the Avalanches returned in 2016 after an ...  
4  We’re going to be stuck with the Chainsmokers ...  
20873
20867


## Keyword Extraction

Use the nltk Rake function (from rake-nltk Python package) to extract important keywords from the reviews. This improves the vectorization by removing punctuation and allows for larger batch size by decreasing size of corpus and individual docs (reviews). 

Furthermore, as a new Rake object is instantiated for every row in the dataframe, I've added the artist name as a stopword when extracting keywords for any given review of an artist. This is because the artist name is likely used frequently in a review of their music, so our similarity scores (and therefore recommendations) will be skewed towards albums by that artist. We want more variety to the recommendation, a user has likely already heard of other albums by the same artist--or can simply Google it. 

In [19]:
artists = sample_df['artist']

sample_df['keywords'] = ""

for idx, row in sample_df.iterrows():
  r = Rake(stopwords=artists[idx])
  r.extract_keywords_from_text(row['review'])
  keywords = list(r.get_word_degrees().keys())
  row['keywords'] = " ".join(keywords)

corpus = sample_df['keywords']

## Vectorization
The working dataframe is set up containing album titles (as index) and the full written review. We're going to vectorize that review column using Tfidf. This will give us a vector of word frequencies across the whole corpus. Think of this as mapping every review onto an n-dimensional space, where n is the number of distinct words in the corpus. Then we can measure similarity between the vectors using cosine similarity.
Admittedly, Tfidf is a little different as it measures frequency relative to how often that word appears in different documents, but this is a good way to conceptualize.


In [20]:
if tfidf:
  tv = TfidfVectorizer()
  tv_counts = tv.fit_transform(corpus)
  # tv_counts = tv.transform(df['review'])
  tv_counts = tv_counts.toarray()
  print(len(tv_counts[0])) # How many words are being counted?

In [21]:
if not tfidf:
    cv = CountVectorizer()
    cv_counts = cv.fit_transform(corpus)
    cv_counts = cv_counts.toarray()
    print(len(cv_counts[0]))

88560


In [22]:
labels = []
albums = sample_df['album']
for i in range(len(tv_counts[0])):
  labels.append("word" + str(i))
counts_df = pd.DataFrame(tv_counts, columns=labels, index=albums)
print(counts_df.head())

                            word0  word1  word2  word3  word4  word5  word6  \
album                                                                         
We the Best                   0.0    0.0    0.0    0.0    0.0    0.0    0.0   
Child Ballads                 0.0    0.0    0.0    0.0    0.0    0.0    0.0   
An Optimist Notes the Dusk    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
Floating World                0.0    0.0    0.0    0.0    0.0    0.0    0.0   
T.I. vs. T.I.P.               0.0    0.0    0.0    0.0    0.0    0.0    0.0   

                            word7  word8  word9  ...  word88809  word88810  \
album                                            ...                         
We the Best                   0.0    0.0    0.0  ...        0.0        0.0   
Child Ballads                 0.0    0.0    0.0  ...        0.0        0.0   
An Optimist Notes the Dusk    0.0    0.0    0.0  ...        0.0        0.0   
Floating World                0.0    0.0    0.0  ...    

## (Cosine) Similarity 
Now we have a dataframe containing the word frequencies and we can compare each album using cosine similarity. Doing this in a pairwise manner (across every row in the dataframe) allows us to measure the similarity between two albums based on their review.

In [23]:
similarity = cosine_similarity(counts_df)
print(similarity[:5])

[[1.         0.05163391 0.07008805 ... 0.05995359 0.05392151 0.05443682]
 [0.05163391 1.         0.04531815 ... 0.0498001  0.04011863 0.0562401 ]
 [0.07008805 0.04531815 1.         ... 0.06610989 0.06276235 0.04078744]
 [0.03617608 0.03489158 0.04751916 ... 0.03875215 0.0355261  0.0291096 ]
 [0.0468151  0.03875305 0.0521827  ... 0.05072077 0.04770401 0.0409124 ]]


God bless, everything ran and nothing has crashed yet (hopefully). Now let's make sense of the similarity matrix. Each album gets a row in the matrix (i.e. album[0] has a corresponding vector at similarity[0]).


## Recommendation and User Testing
Every value in the album vector represents how similar that album is with another. So for album[i], there is a corresponding album vector such that the value at index j in that vector is showing how similar album[i] is with album[j]. If i == j we should see a value of 1, meaning that the two albums are exactly alike (as they should be, it's the same album).

So to find recommendations for a given album, sort the corresponding similarity vector (while maintaining the index). Using the index of the highest value which isn't 1 will reference the album most similar to the given album based on the written Pitchfork review.

The process to recommend is as follows: Given an input album title -> get review corresponding to album, vectorize using Tfidf, measure similarity against sample, return top 5 (or 10?) most similar albums.

In [24]:
working_df.set_index('album', inplace=True) 

In [25]:
search_album = "My Beautiful Dark Twisted Fantasy"
review = working_df.loc[search_album]['review']

vec = tv.transform([review]).toarray()
print(len(vec[0]))
search_sim = cosine_similarity(vec, counts_df)
print(search_sim)

88819
[[0.07803562 0.05944719 0.09819942 ... 0.07605107 0.06006445 0.0644322 ]]


In [26]:
search_df = pd.DataFrame(search_sim[0])
top_5 = search_df.nlargest(5, 0)
if albums.iloc[top_5.index[0]] == search_album:
    top_5 = search_df.nlargest(6, 0)[1:6]
# print(top_5.index)
for idx in top_5.index:
  # print(idx)
  print(albums.iloc[idx])

Sing Me a Song
Sleep Like It’s Winter
3 Feet High and Rising
Speak for Yourself
Chelsea Light Moving


## Results

One thing I wanted to test was the differences in recommendations using the TfidfVectorizer vs. the CountVectorizer. After some basic A/B testing, I found Tfidf to be superior, because the recommendations it gave just made more sense. Given a Kanye West album, I got a mix of albums by him and his label mates recommended back. This is likely because a single reviewer writes the reviews for that related group of artists. The CountVectorizer gave more novel, albeit seemingly random, recommendations that I'm not sure are sensible. I suppose it's up to the user which they prefer.

The state of the art recommendation engine I want to compare my results against is the Spotify engine, which I can easily query via their [web API](https://developer.spotify.com/documentation/web-api/reference/browse/get-recommendations/). Granted, it would be very difficult to create a recommendation engine that is on par with Spotify's due to the amount of data and compute resources they have at their disposal. I would wager that every minute Spotify collects more data on music preference than I would be able to source in a month. Furthermore, their recommender is likely a complex hybrid system that makes use of several models which are actively being trained with new data--however often they are updated. But I digress, the comparison of my recommendations to Spotify's are below. Also note that the recommendations I give are limited to the albums in this dataset, if I could find a way to similarly limit Spotify's recommendation pool I would do so.

In [ ]:
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())
